# 7. Transformation des données

## 7.1 Index et colonnes hiérarchiques
La librairie **pandas** permet de gérer des index et des colonnes hiérarchiques. Dans ce cas, les clefs d'accès sont des tuples (ou bien la clef seule pour le premier niveau).

### 7.1.2 Index hiérarchique ou multi-index
Le *DataFrame* des communes comporte implicitement 3 niveaux d'index : *Région*, *Département* et *Code INSEE* (ou le nom de la commune).

In [1]:
# import des modules usuels
import pandas as pnd
import numpy as np
import matplotlib.pyplot as plt

# commande magique pour l'affichage des graphiques
%matplotlib inline

# options d'affichage
pnd.set_option("display.max_rows", 16)
plt.style.use('seaborn-darkgrid')

In [3]:
# chargement des données
geo = pnd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                  index_col="Code INSEE")
geo.sort_index(inplace=True)
geo.head()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape
Code INSEE,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273..."
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089..."
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190..."
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580..."
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854..."


In [4]:
geo["Latitude"] = geo["geo_point_2d"].apply(lambda x: float(x.split(', ')[0]))
geo["Longitude"] = geo["geo_point_2d"].apply(lambda x: float(x.split(', ')[1]))
geo["Superficie"] /= 100.0
geo["Densité"] = 1000 * geo["Population"] / geo["Superficie"]
geo["Statut"] = geo["Statut"].astype('category',
                                     categories=["Commune simple", "Chef-lieu canton", "Sous-préfecture",
                                                    "Préfecture", "Préfecture de région", "Capitale d'état"],
                                         ordered=True)

In [5]:
geo.index

Index(['01001', '01002', '01004', '01005', '01006', '01007', '01008', '01009',
       '01010', '01011',
       ...
       '97608', '97609', '97610', '97611', '97612', '97613', '97614', '97615',
       '97616', '97617'],
      dtype='object', name='Code INSEE', length=36742)

On annule l'index existant avec la fonction **reset_index()** :

In [6]:
# annulation de l'index existant
geo1 = geo.reset_index()
type(geo1.index)

pandas.core.indexes.range.RangeIndex

In [7]:
# on applique un index triple aux communes
geo1.set_index(["Région", "Département", "Code INSEE"], inplace=True)
geo1

Code Postal                  Commune  \
Région      Département Code INSEE                                        
RHONE-ALPES AIN         01001            01400  L'ABERGEMENT-CLEMENCIAT   
                        01002            01640    L'ABERGEMENT-DE-VAREY   
                        01004            01500        AMBERIEU-EN-BUGEY   
                        01005            01330      AMBERIEUX-EN-DOMBES   
                        01006            01300                  AMBLEON   
                        01007            01500                 AMBRONAY   
                        01008            01500                 AMBUTRIX   
                        01009            01300         ANDERT-ET-CONDON   
...                                        ...                      ...   
MAYOTTE     MAYOTTE     97610            97600                  KOUNGOU   
                        97611            97600                MAMOUDZOU   
                        97612            97630                MTSAMBORO   
                        97613            97650            M'TSANGAMOUJI   
                        97614            97670                 OUANGANI   
                        97615            97610                 PAMANDZI   
                        97616            97640                     SADA   
                        97617            97680                 TSINGONI   

                                              Statut  Altitude Moyenne  \
Région      Département Code INSEE                                       
RHONE-ALPES AIN         01001         Commune simple             242.0   
                        01002         Commune simple             483.0   
                        01004       Chef-lieu canton             379.0   
                        01005         Commune simple             290.0   
                        01006         Commune simple             589.0   
                        01007         Commune simple             309.0   
                        01008         Commune simple             274.0   
                        01009         Commune simple             294.0   
...                                              ...               ...   
MAYOTTE     MAYOTTE     97610       Chef-lieu canton             138.0   
                        97611        Sous-préfecture             161.0   
                        97612       Chef-lieu canton             146.0   
                        97613       Chef-lieu canton              96.0   
                        97614       Chef-lieu canton             175.0   
                        97615       Chef-lieu canton              52.0   
                        97616       Chef-lieu canton             130.0   
                        97617       Chef-lieu canton             147.0   

                                    Superficie  Population  \
Région      Département Code INSEE                           
RHONE-ALPES AIN         01001            15.65         0.8   
                        01002             9.12         0.2   
                        01004            24.48        13.4   
                        01005            16.05         1.6   
                        01006             6.02         0.1   
                        01007            33.59         2.3   
                        01008             5.18         0.7   
                        01009             6.96         0.3   
...                                        ...         ...   
MAYOTTE     MAYOTTE     97610            27.69        19.8   
                        97611            42.35        53.0   
                        97612            15.88         6.9   
                        97613            21.55         5.0   
                        97614            18.28         6.6   
                        97615             4.26         9.1   
                        97616            10.85         8.0   
                        97617            34.26         9.2   

                       

In [8]:
# l'index est devenu un MultiIndex
type(geo1.index)

pandas.core.indexes.multi.MultiIndex

Accès direct au premier niveau :

In [9]:
geo1.loc[("ILE-DE-FRANCE")]  # ou bien communes1.loc["ILE-DE-FRANCE"]

Code Postal                   Commune  \
Département Code INSEE                                         
PARIS       75101            75001  PARIS-1ER-ARRONDISSEMENT   
            75102            75002   PARIS-2E-ARRONDISSEMENT   
            75103            75003   PARIS-3E-ARRONDISSEMENT   
            75104            75004   PARIS-4E-ARRONDISSEMENT   
            75105            75005   PARIS-5E-ARRONDISSEMENT   
            75106            75006   PARIS-6E-ARRONDISSEMENT   
            75107            75007   PARIS-7E-ARRONDISSEMENT   
            75108            75008   PARIS-8E-ARRONDISSEMENT   
...                            ...                       ...   
VAL-D'OISE  95658            95450                     VIGNY   
            95660            95570       VILLAINES-SOUS-BOIS   
            95675            95380                  VILLERON   
            95676            95510        VILLERS-EN-ARTHIES   
            95678            95840             VILLIERS-ADAM   
            95680            95400           VILLIERS-LE-BEL   
            95682            95720           VILLIERS-LE-SEC   
            95690            95420       WY-DIT-JOLI-VILLAGE   

                                  Statut  Altitude Moyenne  Superficie  \
Département Code INSEE                                                   
PARIS       75101        Capitale d'état              33.0        1.81   
            75102       Chef-lieu canton              36.0        0.99   
            75103       Chef-lieu canton              35.0        1.16   
            75104       Chef-lieu canton              33.0        1.60   
            75105       Chef-lieu canton              42.0        2.52   
            75106       Chef-lieu canton              40.0        2.15   
            75107       Chef-lieu canton              34.0        4.11   
            75108       Chef-lieu canton              39.0        3.85   
...                                  ...               ...         ...   
VAL-D'OISE  95658       Chef-lieu canton             103.0        6.81   
            95660         Commune simple             120.0        1.90   
            95675         Commune simple             115.0        5.71   
            95676         Commune simple             161.0        8.42   
            95678         Commune simple             104.0        9.87   
            95680       Chef-lieu canton              87.0        7.29   
            95682         Commune simple             127.0        3.19   
            95690         Commune simple             135.0        8.43   

                        Population                  geo_point_2d  \
Département Code INSEE                                             
PARIS       75101             17.6  48.8626304852, 2.33629344655   
            75102             22.4  48.8679033789, 2.34410716666   
            75103             35.7  48.8630541318, 2.35936105897   
            75104             28.2   48.854228282, 2.35736193814   
            75105             61.5  48.8445086596, 2.34985938556   
            75106             43.1  48.8489680919, 2.33267089859   
            75107             57.4  48.8560825982, 2.31243868773   
            75108             40.3  48.8725272666, 2.31258256042   
...                            ...                           ...   
VAL-D'OISE  95658              1.1   49.080165557, 1.92992323682   
            95660              0.7  49.0753739915, 2.35663670214   
            95675              0.7    49.060842982, 2.5345109745   
            95676              0.5  49.0858999284, 1.73039629888   
            95678              0.8  49.0702888248, 2.23950871871   
            95680             27.0  49.0084494157, 2.40385604039   
            95682              0.2  49.0743088682, 2.38688959984   
            95690              0.3  49.0958197758, 1.82502883308   

                                                                geo_shape  \
Département Code INSEE       

Accès au deuxième niveau :

In [10]:
geo1.loc[("ILE-DE-FRANCE", "HAUTS-DE-SEINE")]

C:\Test\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  if __name__ == '__main__':


,Code Postal,Commune,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,Densité
Code INSEE,,,,,,,,,,,
92002,92160,ANTONY,Sous-préfecture,66.0,9.56,61.4,"48.7503412602, 2.2993268102","{""type"": ""Polygon"", ""coordinates"": [[[2.303837...",48.750341,2.299327,6422.594142
92004,92600,ASNIERES-SUR-SEINE,Chef-lieu canton,31.0,4.82,81.6,"48.9153530123, 2.2880384663","{""type"": ""Polygon"", ""coordinates"": [[[2.284270...",48.915353,2.288038,16929.460581
92007,92220,BAGNEUX,Chef-lieu canton,86.0,4.18,38.5,"48.7983229866, 2.30989995212","{""type"": ""Polygon"", ""coordinates"": [[[2.309699...",48.798323,2.309900,9210.526316
92009,92270,BOIS-COLOMBES,Chef-lieu canton,36.0,1.93,28.2,"48.9153368426, 2.26738552597","{""type"": ""Polygon"", ""coordinates"": [[[2.266398...",48.915337,2.267386,14611.398964
92012,92100,BOULOGNE-BILLANCOURT,Sous-préfecture,33.0,6.15,113.1,"48.8365843138, 2.23913599058","{""type"": ""Polygon"", ""coordinates"": [[[2.236100...",48.836584,2.239136,18390.243902
92014,92340,BOURG-LA-REINE,Chef-lieu canton,55.0,1.86,19.8,"48.7799073627, 2.31643010763","{""type"": ""Polygon"", ""coordinates"": [[[2.319915...",48.779907,2.316430,10645.161290
92019,92290,CHATENAY-MALABRY,Chef-lieu canton,126.0,6.37,32.4,"48.7681690197, 2.26282598525","{""type"": ""Polygon"", ""coordinates"": [[[2.276145...",48.768169,2.262826,5086.342229
92020,92320,CHATILLON,Chef-lieu canton,101.0,2.93,32.4,"48.8034091756, 2.28799131883","{""type"": ""Polygon"", ""coordinates"": [[[2.272195...",48.803409,2.287991,11058.020478
...,...,...,...,...,...,...,...,...,...,...,...


Accès au troisième niveau :

In [11]:
geo1.loc[("ILE-DE-FRANCE", "HAUTS-DE-SEINE", "92002")]

Code Postal                                                     92160
Commune                                                        ANTONY
Statut                                                Sous-préfecture
Altitude Moyenne                                                   66
Superficie                                                       9.56
Population                                                       61.4
geo_point_2d                              48.7503412602, 2.2993268102
geo_shape           {"type": "Polygon", "coordinates": [[[2.303837...
Latitude                                                      48.7503
Longitude                                                     2.29933
Densité                                                       6422.59
Name: (ILE-DE-FRANCE, HAUTS-DE-SEINE, 92002), dtype: object

Tri du multi-index pour éviter les problèmes de performance.

In [12]:
geo1.sort_index(inplace=True)
geo1.loc[("ILE-DE-FRANCE", "HAUTS-DE-SEINE")]

,Code Postal,Commune,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,Densité
Code INSEE,,,,,,,,,,,
92002,92160,ANTONY,Sous-préfecture,66.0,9.56,61.4,"48.7503412602, 2.2993268102","{""type"": ""Polygon"", ""coordinates"": [[[2.303837...",48.750341,2.299327,6422.594142
92004,92600,ASNIERES-SUR-SEINE,Chef-lieu canton,31.0,4.82,81.6,"48.9153530123, 2.2880384663","{""type"": ""Polygon"", ""coordinates"": [[[2.284270...",48.915353,2.288038,16929.460581
92007,92220,BAGNEUX,Chef-lieu canton,86.0,4.18,38.5,"48.7983229866, 2.30989995212","{""type"": ""Polygon"", ""coordinates"": [[[2.309699...",48.798323,2.309900,9210.526316
92009,92270,BOIS-COLOMBES,Chef-lieu canton,36.0,1.93,28.2,"48.9153368426, 2.26738552597","{""type"": ""Polygon"", ""coordinates"": [[[2.266398...",48.915337,2.267386,14611.398964
92012,92100,BOULOGNE-BILLANCOURT,Sous-préfecture,33.0,6.15,113.1,"48.8365843138, 2.23913599058","{""type"": ""Polygon"", ""coordinates"": [[[2.236100...",48.836584,2.239136,18390.243902
92014,92340,BOURG-LA-REINE,Chef-lieu canton,55.0,1.86,19.8,"48.7799073627, 2.31643010763","{""type"": ""Polygon"", ""coordinates"": [[[2.319915...",48.779907,2.316430,10645.161290
92019,92290,CHATENAY-MALABRY,Chef-lieu canton,126.0,6.37,32.4,"48.7681690197, 2.26282598525","{""type"": ""Polygon"", ""coordinates"": [[[2.276145...",48.768169,2.262826,5086.342229
92020,92320,CHATILLON,Chef-lieu canton,101.0,2.93,32.4,"48.8034091756, 2.28799131883","{""type"": ""Polygon"", ""coordinates"": [[[2.272195...",48.803409,2.287991,11058.020478
...,...,...,...,...,...,...,...,...,...,...,...


### Colonnes hiérarchiques

In [14]:
thd = pnd.read_excel("FranceTHD_Open_Data_Observatoire_Juin2015.xlsx",
                    sheetname="Communes",
                    header=1,
                    index_col="Code INSEE",
                    names=["Département", "Commune",
                                "1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit",
                                "DSL 1 Mbit", "DSL 3 Mbit", "DSL 8 Mbit", "DSL 30 Mbit", "DSL 100 Mbit",
                                "Câble 1 Mbit", "Câble 3 Mbit", "Câble 8 Mbit", "Câble 30 Mbit", "Câble 100 Mbit",
                                "Fibre 1 Mbit", "Fibre 3 Mbit", "Fibre 8 Mbit", "Fibre 30 Mbit", "Fibre 100 Mbit"])
thd.head()

,Département,Commune,1 Mbit,3 Mbit,8 Mbit,30 Mbit,100 Mbit,DSL 1 Mbit,DSL 3 Mbit,DSL 8 Mbit,...,Câble 1 Mbit,Câble 3 Mbit,Câble 8 Mbit,Câble 30 Mbit,Câble 100 Mbit,Fibre 1 Mbit,Fibre 3 Mbit,Fibre 8 Mbit,Fibre 30 Mbit,Fibre 100 Mbit
Code INSEE,,,,,,,,,,,,,,,,,,,,,
01001,AIN,L'Abergement-Clémenciat,1.000,0.448,0.052,0.000,0.000,1.000,0.448,0.052,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01002,AIN,L'Abergement-de-Varey,0.676,0.594,0.571,0.571,0.571,0.124,0.024,0.000,...,0.0,0.0,0.0,0.0,0.0,0.571,0.571,0.571,0.571,0.571
01004,AIN,Ambérieu-en-Bugey,1.000,0.966,0.794,0.234,0.000,1.000,0.966,0.794,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01005,AIN,Ambérieux-en-Dombes,1.000,0.990,0.942,0.667,0.004,1.000,0.985,0.937,...,0.0,0.0,0.0,0.0,0.0,0.004,0.004,0.004,0.004,0.004
01006,AIN,Ambléon,1.000,1.000,1.000,0.934,0.934,1.000,1.000,1.000,...,0.0,0.0,0.0,0.0,0.0,0.934,0.934,0.934,0.934,0.934


In [15]:
# on restreint les données au haut débit seul
thd2 = thd.loc[:, "1 Mbit":]
thd2.head()

,1 Mbit,3 Mbit,8 Mbit,30 Mbit,100 Mbit,DSL 1 Mbit,DSL 3 Mbit,DSL 8 Mbit,DSL 30 Mbit,DSL 100 Mbit,Câble 1 Mbit,Câble 3 Mbit,Câble 8 Mbit,Câble 30 Mbit,Câble 100 Mbit,Fibre 1 Mbit,Fibre 3 Mbit,Fibre 8 Mbit,Fibre 30 Mbit,Fibre 100 Mbit
Code INSEE,,,,,,,,,,,,,,,,,,,,
01001,1.000,0.448,0.052,0.000,0.000,1.000,0.448,0.052,0.000,0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01002,0.676,0.594,0.571,0.571,0.571,0.124,0.024,0.000,0.000,0,0.0,0.0,0.0,0.0,0.0,0.571,0.571,0.571,0.571,0.571
01004,1.000,0.966,0.794,0.234,0.000,1.000,0.966,0.794,0.234,0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01005,1.000,0.990,0.942,0.667,0.004,1.000,0.985,0.937,0.663,0,0.0,0.0,0.0,0.0,0.0,0.004,0.004,0.004,0.004,0.004
01006,1.000,1.000,1.000,0.934,0.934,1.000,1.000,1.000,0.000,0,0.0,0.0,0.0,0.0,0.0,0.934,0.934,0.934,0.934,0.934


On constate qu'il y a implicitement 2 niveaux de colonnes : Techno et Débit (ce qu'il y avait dans le fichier Excel initial).

In [16]:
# on crée des colonnes hiérarchiques que l'on nomme également
thd2.columns = [["THD", "THD", "THD", "THD", "THD",
                     "DSL", "DSL", "DSL", "DSL", "DSL",
                     "Câble", "Câble", "Câble", "Câble", "Câble",
                     "Fibre", "Fibre", "Fibre", "Fibre", "Fibre"],
                   ["1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit",
                   "1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit",
                   "1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit",
                   "1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit"]]
thd2.columns.names = ["Techno", "Débit"]  # on nomme les 2 index"
thd2.head()

Techno        THD                                   DSL                        \
Débit      1 Mbit 3 Mbit 8 Mbit 30 Mbit 100 Mbit 1 Mbit 3 Mbit 8 Mbit 30 Mbit   
Code INSEE                                                                      
01001       1.000  0.448  0.052   0.000    0.000  1.000  0.448  0.052   0.000   
01002       0.676  0.594  0.571   0.571    0.571  0.124  0.024  0.000   0.000   
01004       1.000  0.966  0.794   0.234    0.000  1.000  0.966  0.794   0.234   
01005       1.000  0.990  0.942   0.667    0.004  1.000  0.985  0.937   0.663   
01006       1.000  1.000  1.000   0.934    0.934  1.000  1.000  1.000   0.000   

Techno               Câble                                 Fibre         \
Débit      100 Mbit 1 Mbit 3 Mbit 8 Mbit 30 Mbit 100 Mbit 1 Mbit 3 Mbit   
Code INSEE                                                                
01001             0    0.0    0.0    0.0     0.0      0.0  0.000  0.000   
01002             0    0.0    0.0    0.0     0.0      0.0  0.571  0.571   
01004             0    0.0    0.0    0.0     0.0      0.0  0.000  0.000   
01005             0    0.0    0.0    0.0     0.0      0.0  0.004  0.004   
01006             0    0.0    0.0    0.0     0.0      0.0  0.934  0.934   

Techno                              
Débit      8 Mbit 30 Mbit 100 Mbit  
Code INSEE                          
01001       0.000   0.000    0.000  
01002       0.571   0.571    0.571  
01004       0.000   0.000    0.000  
01005       0.004   0.004    0.004  
01006       0.934   0.934    0.934

On retrouve la structure du fichier Excel initial avec des cellules fusionnées.

In [17]:
# accès à une valeur
thd2.loc["01002", ("DSL", "3 Mbit")]

0.024

In [18]:
thd2["DSL"].columns

Index(['1 Mbit', '3 Mbit', '8 Mbit', '30 Mbit', '100 Mbit'], dtype='object', name='Débit')

In [19]:
# accès à une portion de ligne
thd2.loc["01002", ("DSL")]  # ou thd2.loc["01001", "DSL"]

Débit
1 Mbit      0.124
3 Mbit      0.024
8 Mbit      0.000
30 Mbit     0.000
100 Mbit    0.000
Name: 01002, dtype: float64

**Remarque** : La liste des colonnes hiérarchiques comporte des répétitions. Il aurait été possible d'utiliser les opérateurs *+* et *\** qui agissent sur les listes :

In [20]:
[["THD"] * 5 + ["DSL"] * 5 + ["Câble"] * 5 + ["Fibre"] * 5,
 ["1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit"] * 4]

[['THD',
  'THD',
  'THD',
  'THD',
  'THD',
  'DSL',
  'DSL',
  'DSL',
  'DSL',
  'DSL',
  'Câble',
  'Câble',
  'Câble',
  'Câble',
  'Câble',
  'Fibre',
  'Fibre',
  'Fibre',
  'Fibre',
  'Fibre'],
 ['1 Mbit',
  '3 Mbit',
  '8 Mbit',
  '30 Mbit',
  '100 Mbit',
  '1 Mbit',
  '3 Mbit',
  '8 Mbit',
  '30 Mbit',
  '100 Mbit',
  '1 Mbit',
  '3 Mbit',
  '8 Mbit',
  '30 Mbit',
  '100 Mbit',
  '1 Mbit',
  '3 Mbit',
  '8 Mbit',
  '30 Mbit',
  '100 Mbit']]

### Permutation de niveaux hiérarchiques

Il est possible d'échanger des niveaux hiérarchiques des index ou des colonnes avec la fonction **swaplevel()** :

In [22]:
thd2.swaplevel("Débit", "Techno", axis=1).head()

Débit,1 Mbit,3 Mbit,8 Mbit,30 Mbit,100 Mbit,1 Mbit,3 Mbit,8 Mbit,30 Mbit,100 Mbit,1 Mbit,3 Mbit,8 Mbit,30 Mbit,100 Mbit,1 Mbit,3 Mbit,8 Mbit,30 Mbit,100 Mbit
Techno,THD,THD,THD,THD,THD,DSL,DSL,DSL,DSL,DSL,Câble,Câble,Câble,Câble,Câble,Fibre,Fibre,Fibre,Fibre,Fibre
Code INSEE,,,,,,,,,,,,,,,,,,,,
01001,1.000,0.448,0.052,0.000,0.000,1.000,0.448,0.052,0.000,0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01002,0.676,0.594,0.571,0.571,0.571,0.124,0.024,0.000,0.000,0,0.0,0.0,0.0,0.0,0.0,0.571,0.571,0.571,0.571,0.571
01004,1.000,0.966,0.794,0.234,0.000,1.000,0.966,0.794,0.234,0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01005,1.000,0.990,0.942,0.667,0.004,1.000,0.985,0.937,0.663,0,0.0,0.0,0.0,0.0,0.0,0.004,0.004,0.004,0.004,0.004
01006,1.000,1.000,1.000,0.934,0.934,1.000,1.000,1.000,0.000,0,0.0,0.0,0.0,0.0,0.0,0.934,0.934,0.934,0.934,0.934


Les niveaux sont bien inversés mais les débits identiques sont éclatés. On verra ci-après comment les rassembler.

## 7.2 Opérations de pivot / Tableaux croisés

La méthode **pivot_table()** construit un tableau synthétique de valeurs agrégées et ventilées selon les différentes valeurs d'une ou plusieurs colonnes.

Elle retourne un nouveau *DataFrame* en fonction des paramètres fournis.

- index : colonnes du *DataFrame* initial dont les valeurs sont utilisées en index
- values : colonnes du *DataFrame* initial dont les valeurs sont agrégées
- columns : colonnes du *DataFrame* initial dont les valeurs sont utilisées en nom de colonnes
- aggfunc : fonction d'agrégation des valeurs, par défaut **numpy.mean** (calcul de la moyenne des valeurs)
- margins : avec ou sans calcul des sous-totaux

Il existe également la fonction **crosstab()** qui calcule par défaut la table des décomptes ou des fréquences croisées.

In [23]:
# tableau croisé" "Statut" x "Altitude Moyenne", "Population", "Superficie"
geo.pivot_table(index="Statut", values=["Altitude Moyenne", "Population", "Superficie"])

,Altitude Moyenne,Population,Superficie
Statut,,,
Commune simple,279.097813,0.868631,14.665004
Chef-lieu canton,269.461074,7.381477,43.604453
Sous-préfecture,236.016667,22.865417,56.935750
Préfecture,220.905405,58.458108,37.894189
Préfecture de région,141.884615,140.057692,47.372692
Capitale d'état,33.000000,17.600000,1.810000


In [24]:
# tableau croisé" "Région" x "Statut" avec la somme de "Population"
geo.pivot_table(index="Région", columns="Statut", values="Population", aggfunc='sum')

Statut,Commune simple,Chef-lieu canton,Sous-préfecture,Préfecture,Préfecture de région,Capitale d'état
Région,,,,,,
ALSACE,973.2,307.9,224.3,67.2,271.7,NaN
AQUITAINE,1672.6,902.5,219.3,176.4,236.7,NaN
AUVERGNE,677.0,323.6,138.5,66.8,138.6,NaN
BASSE-NORMANDIE,861.7,332.8,122.3,46.1,109.3,NaN
BOURGOGNE,882.5,351.9,147.5,108.3,152.1,NaN
BRETAGNE,1697.5,760.7,348.0,162.1,206.6,NaN
CENTRE,1258.4,642.6,190.7,333.5,113.2,NaN
CHAMPAGNE-ARDENNE,662.9,197.3,299.7,134.6,46.2,NaN
...,...,...,...,...,...,...


In [25]:
# fréquences croisées
pnd.set_option('display.precision', 3)
pnd.crosstab(geo["Région"], geo["Statut"], normalize=True, margins=True) * 100

Statut,Commune simple,Chef-lieu canton,Sous-préfecture,Préfecture,Préfecture de région,Capitale d'état,All
Région,,,,,,,
ALSACE,2.292,0.136,0.027,0.003,0.003,0.000,2.460
AQUITAINE,5.696,0.501,0.038,0.011,0.003,0.000,6.249
AUVERGNE,3.201,0.327,0.027,0.008,0.003,0.000,3.565
BASSE-NORMANDIE,4.600,0.302,0.022,0.005,0.003,0.000,4.932
BOURGOGNE,5.166,0.362,0.030,0.008,0.003,0.000,5.569
BRETAGNE,2.994,0.422,0.030,0.008,0.003,0.000,3.457
CENTRE,4.556,0.400,0.038,0.014,0.003,0.000,5.011
CHAMPAGNE-ARDENNE,5.000,0.278,0.030,0.008,0.003,0.000,5.318
...,...,...,...,...,...,...,...


Les méthodes **stack()** et **unstack()** permettent d'empiler les colonnes en index et de dépiler les index en colonnes.

In [26]:
thd2.head()

Techno        THD                                   DSL                        \
Débit      1 Mbit 3 Mbit 8 Mbit 30 Mbit 100 Mbit 1 Mbit 3 Mbit 8 Mbit 30 Mbit   
Code INSEE                                                                      
01001       1.000  0.448  0.052   0.000    0.000  1.000  0.448  0.052   0.000   
01002       0.676  0.594  0.571   0.571    0.571  0.124  0.024  0.000   0.000   
01004       1.000  0.966  0.794   0.234    0.000  1.000  0.966  0.794   0.234   
01005       1.000  0.990  0.942   0.667    0.004  1.000  0.985  0.937   0.663   
01006       1.000  1.000  1.000   0.934    0.934  1.000  1.000  1.000   0.000   

Techno               Câble                                 Fibre         \
Débit      100 Mbit 1 Mbit 3 Mbit 8 Mbit 30 Mbit 100 Mbit 1 Mbit 3 Mbit   
Code INSEE                                                                
01001             0    0.0    0.0    0.0     0.0      0.0  0.000  0.000   
01002             0    0.0    0.0    0.0     0.0      0.0  0.571  0.571   
01004             0    0.0    0.0    0.0     0.0      0.0  0.000  0.000   
01005             0    0.0    0.0    0.0     0.0      0.0  0.004  0.004   
01006             0    0.0    0.0    0.0     0.0      0.0  0.934  0.934   

Techno                              
Débit      8 Mbit 30 Mbit 100 Mbit  
Code INSEE                          
01001       0.000   0.000    0.000  
01002       0.571   0.571    0.571  
01004       0.000   0.000    0.000  
01005       0.004   0.004    0.004  
01006       0.934   0.934    0.934

La méthode **stack()** prend le niveau de colonne le plus bas et l'envoie au niveau d'index le plus bas.

In [27]:
# on déplace le niveau de colonne "Débit" vers les index
thd2.stack()

Techno               Câble    DSL  Fibre    THD
Code INSEE Débit                               
01001      1 Mbit      0.0  1.000  0.000  1.000
           100 Mbit    0.0  0.000  0.000  0.000
           3 Mbit      0.0  0.448  0.000  0.448
           30 Mbit     0.0  0.000  0.000  0.000
           8 Mbit      0.0  0.052  0.000  0.052
01002      1 Mbit      0.0  0.124  0.571  0.676
           100 Mbit    0.0  0.000  0.571  0.571
           3 Mbit      0.0  0.024  0.571  0.594
...                    ...    ...    ...    ...
97701      3 Mbit      0.0  0.907  0.000  0.907
           30 Mbit     0.0  0.136  0.000  0.136
           8 Mbit      0.0  0.745  0.000  0.745
97801      1 Mbit      0.0  0.993  0.000  0.993
           100 Mbit    0.0  0.000  0.000  0.000
           3 Mbit      0.0  0.787  0.000  0.787
           30 Mbit     0.0  0.211  0.000  0.211
           8 Mbit      0.0  0.663  0.000  0.663

[183465 rows x 4 columns]

On peut répéter l'opération jusqu'à épuisement des niveaux de colonnes. A la fin on obtient un objet *Series* avec un index hiérarchique.

In [28]:
# on déplace les niveau de colonne "Débit" et "Techno" vers les index
thd2.stack().stack()

Code INSEE  Débit     Techno
01001       1 Mbit    Câble     0.000
                      DSL       1.000
                      Fibre     0.000
                      THD       1.000
            100 Mbit  Câble     0.000
                      DSL       0.000
                      Fibre     0.000
                      THD       0.000
                                ...  
97801       30 Mbit   Câble     0.000
                      DSL       0.211
                      Fibre     0.000
                      THD       0.211
            8 Mbit    Câble     0.000
                      DSL       0.663
                      Fibre     0.000
                      THD       0.663
Length: 733860, dtype: float64

Inversement, la méthode **unstack()** prend le niveau d'index le plus bas et l'envoie au niveau de colonne le plus bas.

In [29]:
# unstack
thd2.stack().unstack()

Techno      Câble                                   DSL                  \
Débit      1 Mbit 100 Mbit 3 Mbit 30 Mbit 8 Mbit 1 Mbit 100 Mbit 3 Mbit   
Code INSEE                                                                
01001         0.0      0.0    0.0     0.0    0.0  1.000      0.0  0.448   
01002         0.0      0.0    0.0     0.0    0.0  0.124      0.0  0.024   
01004         0.0      0.0    0.0     0.0    0.0  1.000      0.0  0.966   
01005         0.0      0.0    0.0     0.0    0.0  1.000      0.0  0.985   
01006         0.0      0.0    0.0     0.0    0.0  1.000      0.0  1.000   
01007         0.0      0.0    0.0     0.0    0.0  0.778      0.0  0.066   
01008         0.0      0.0    0.0     0.0    0.0  1.000      0.0  0.557   
01009         0.0      0.0    0.0     0.0    0.0  1.000      0.0  0.616   
...           ...      ...    ...     ...    ...    ...      ...    ...   
97612         0.0      0.0    0.0     0.0    0.0  0.996      0.0  0.871   
97613         0.0      0.0    0.0     0.0    0.0  0.999      0.0  0.937   
97614         0.0      0.0    0.0     0.0    0.0  1.000      0.0  0.972   
97615         0.0      0.0    0.0     0.0    0.0  1.000      0.0  1.000   
97616         0.0      0.0    0.0     0.0    0.0  0.991      0.0  0.927   
97617         0.0      0.0    0.0     0.0    0.0  0.990      0.0  0.709   
97701         0.0      0.0    0.0     0.0    0.0  1.000      0.0  0.907   
97801         0.0      0.0    0.0     0.0    0.0  0.993      0.0  0.787   

Techno                     Fibre                                   THD  \
Débit      30 Mbit 8 Mbit 1 Mbit 100 Mbit 3 Mbit 30 Mbit 8 Mbit 1 Mbit   
Code INSEE                                                               
01001        0.000  0.052  0.000    0.000  0.000   0.000  0.000  1.000   
01002        0.000  0.000  0.571    0.571  0.571   0.571  0.571  0.676   
01004        0.234  0.794  0.000    0.000  0.000   0.000  0.000  1.000   
01005        0.663  0.937  0.004    0.004  0.004   0.004  0.004  1.000   
01006        0.000  1.000  0.934    0.934  0.934   0.934  0.934  1.000   
01007        0.000  0.059  0.713    0.713  0.713   0.713  0.713  0.973   
01008        0.000  0.013  0.000    0.000  0.000   0.000  0.000  1.000   
01009        0.000  0.584  0.000    0.000  0.000   0.000  0.000  1.000   
...            ...    ...    ...      ...    ...     ...    ...    ...   
97612        0.394  0.624  0.000    0.000  0.000   0.000  0.000  0.996   
97613        0.144  0.885  0.000    0.000  0.000   0.000  0.000  0.999   
97614        0.308  0.762  0.000    0.000  0.000   0.000  0.000  1.000   
97615        0.538  0.998  0.000    0.000  0.000   0.000  0.000  1.000   
97616        0.106  0.765  0.000    0.000  0.000   0.000  0.000  0.991   
97617        0.509  0.707  0.000    0.000  0.000   0.000  0.000  0.990   
97701        0.136  0.745  0.000    0.000  0.000   0.000  0.000  1.000   
97801        0.211  0.663  0.000    0.000  0.000   0.000  0.000  0.993   

Techno                                     
Débit      100 Mbit 3 Mbit 30 Mbit 8 Mbit  
Code INSEE                                 
01001         0.000  0.448   0.000  0.052  
01002         0.571  0.594   0.571  0.571  
01004         0.000  0.966   0.234  0.794  
01005         0.004  0.990   0.667  0.942  
01006         0.934  1.000   0.934  1.000  
01007         0.713  0.779   0.713  0.772  
01008         0.000  0.557   0.000  0.013  
01009         0.000  0.616   0.000  0.584  
...             ...    ...     ...    ...  
97612         0.000  0.871   0.394  0.624  
97613         0.000  0.937   0.144  0.885  
97614         0.000  0.972   0.308  0.762  
97615         0.000  1.000   0.538  0.998  
97616         0.000  0.927   0.106  0.765  
97617         0.000  0.709   0.509  0.707  
97701         0.000  0.907   0.136  0.745  
97801         0.000  0.787   0.211  0.663  

[36693 rows x 20 columns]

Il est possible de préciser le niveau que l'on souhaite empiler ou dépiler : soit en inidiquant le nom de l'index ou de la colonne soit en précisant son rang.

In [30]:
thd2.stack("Techno")  # ou bien thd2.stack(level=0)

Débit              1 Mbit  100 Mbit  3 Mbit  30 Mbit  8 Mbit
Code INSEE Techno                                           
01001      Câble    0.000     0.000   0.000    0.000   0.000
           DSL      1.000     0.000   0.448    0.000   0.052
           Fibre    0.000     0.000   0.000    0.000   0.000
           THD      1.000     0.000   0.448    0.000   0.052
01002      Câble    0.000     0.000   0.000    0.000   0.000
           DSL      0.124     0.000   0.024    0.000   0.000
           Fibre    0.571     0.571   0.571    0.571   0.571
           THD      0.676     0.571   0.594    0.571   0.571
...                   ...       ...     ...      ...     ...
97701      Câble    0.000     0.000   0.000    0.000   0.000
           DSL      1.000     0.000   0.907    0.136   0.745
           Fibre    0.000     0.000   0.000    0.000   0.000
           THD      1.000     0.000   0.907    0.136   0.745
97801      Câble    0.000     0.000   0.000    0.000   0.000
           DSL      0.993     0.000   0.787    0.211   0.663
           Fibre    0.000     0.000   0.000    0.000   0.000
           THD      0.993     0.000   0.787    0.211   0.663

[146772 rows x 5 columns]

En utilant ensuite la méthode *unstack()* on obtient cette fois-ci un tableau avec les débits rassemblés. 

In [31]:
thd2.stack("Techno").unstack()

Débit      1 Mbit                      100 Mbit                    3 Mbit  \
Techno      Câble    DSL  Fibre    THD    Câble  DSL  Fibre    THD  Câble   
Code INSEE                                                                  
01001         0.0  1.000  0.000  1.000      0.0  0.0  0.000  0.000    0.0   
01002         0.0  0.124  0.571  0.676      0.0  0.0  0.571  0.571    0.0   
01004         0.0  1.000  0.000  1.000      0.0  0.0  0.000  0.000    0.0   
01005         0.0  1.000  0.004  1.000      0.0  0.0  0.004  0.004    0.0   
01006         0.0  1.000  0.934  1.000      0.0  0.0  0.934  0.934    0.0   
01007         0.0  0.778  0.713  0.973      0.0  0.0  0.713  0.713    0.0   
01008         0.0  1.000  0.000  1.000      0.0  0.0  0.000  0.000    0.0   
01009         0.0  1.000  0.000  1.000      0.0  0.0  0.000  0.000    0.0   
...           ...    ...    ...    ...      ...  ...    ...    ...    ...   
97612         0.0  0.996  0.000  0.996      0.0  0.0  0.000  0.000    0.0   
97613         0.0  0.999  0.000  0.999      0.0  0.0  0.000  0.000    0.0   
97614         0.0  1.000  0.000  1.000      0.0  0.0  0.000  0.000    0.0   
97615         0.0  1.000  0.000  1.000      0.0  0.0  0.000  0.000    0.0   
97616         0.0  0.991  0.000  0.991      0.0  0.0  0.000  0.000    0.0   
97617         0.0  0.990  0.000  0.990      0.0  0.0  0.000  0.000    0.0   
97701         0.0  1.000  0.000  1.000      0.0  0.0  0.000  0.000    0.0   
97801         0.0  0.993  0.000  0.993      0.0  0.0  0.000  0.000    0.0   

Débit                           30 Mbit                      8 Mbit         \
Techno        DSL  Fibre    THD   Câble    DSL  Fibre    THD  Câble    DSL   
Code INSEE                                                                   
01001       0.448  0.000  0.448     0.0  0.000  0.000  0.000    0.0  0.052   
01002       0.024  0.571  0.594     0.0  0.000  0.571  0.571    0.0  0.000   
01004       0.966  0.000  0.966     0.0  0.234  0.000  0.234    0.0  0.794   
01005       0.985  0.004  0.990     0.0  0.663  0.004  0.667    0.0  0.937   
01006       1.000  0.934  1.000     0.0  0.000  0.934  0.934    0.0  1.000   
01007       0.066  0.713  0.779     0.0  0.000  0.713  0.713    0.0  0.059   
01008       0.557  0.000  0.557     0.0  0.000  0.000  0.000    0.0  0.013   
01009       0.616  0.000  0.616     0.0  0.000  0.000  0.000    0.0  0.584   
...           ...    ...    ...     ...    ...    ...    ...    ...    ...   
97612       0.871  0.000  0.871     0.0  0.394  0.000  0.394    0.0  0.624   
97613       0.937  0.000  0.937     0.0  0.144  0.000  0.144    0.0  0.885   
97614       0.972  0.000  0.972     0.0  0.308  0.000  0.308    0.0  0.762   
97615       1.000  0.000  1.000     0.0  0.538  0.000  0.538    0.0  0.998   
97616       0.927  0.000  0.927     0.0  0.106  0.000  0.106    0.0  0.765   
97617       0.709  0.000  0.709     0.0  0.509  0.000  0.509    0.0  0.707   
97701       0.907  0.000  0.907     0.0  0.136  0.000  0.136    0.0  0.745   
97801       0.787  0.000  0.787     0.0  0.211  0.000  0.211    0.0  0.663   

Débit                     
Techno      Fibre    THD  
Code INSEE                
01001       0.000  0.052  
01002       0.571  0.571  
01004       0.000  0.794  
01005       0.004  0.942  
01006       0.934  1.000  
01007       0.713  0.772  
01008       0.000  0.013  
01009       0.000  0.584  
...           ...    ...  
97612       0.000  0.624  
97613       0.000  0.885  
97614       0.000  0.762  
97615       0.000  0.998  
97616       0.000  0.765  
97617       0.000  0.707  
97701       0.000  0.745  
97801       0.000  0.663  

[36693 rows x 20 columns]

**Exercice dirigé optionnel de mise à plat des données**

1) Repartir du *DataFrame* thd et effectuer un reset de l'index sur une copie

2) Transférer les colonnes *Code INSEE*, *Département* et *Commune* vers l'index.

3) Le reste des colonnes correspond aux données télécom. Créer un index hiérarchique sur les colonnes restantes avec les niveaux *Techno* et *Débit*. On obtient un *DataFrame* avec un index hiérarchique à 3 niveaux et des colonnes sur 2 niveaux.

4) Transférer les colonnes hiérarchiques vers l'index. On obtient un objet Series avec un index hiérarchique à 5 niveaux. Les valeurs sont les débits.

5) Effectuer un reset de l'index et renommer les colonnes *Code INSEE*, *Département*, *Commune*, *Techno* et *Débit* ainsi qu'en ajoutant le nom *Valeur* pour la colonne avec les débits.  On obtient une table avec 6 colonnes distinctes et une ligne par valeur numérique de débit.

6) Il est ensuite possible d'accéder à un débit particulier en précisant la commune, la techno et le débit.

In [ ]:
# suppresion de l'index


La mise à plat des données ne concerne que les données portant sur le haut-débit. On commence par transférer vers l'index les autres colonnes. On obtient un index hiérarchique à 2 niveaux.

In [ ]:
# transfert des autres colonnes vers l'index


Création d'un index hiérarchique sur les colonnes restantes. On obtient un DataFrame avec un index hiérarchique à 3 niveaux et des colonnes sur 2 niveaux.

In [ ]:
# création d'un index héirarchique


Transfert des colonnes hiérarchiques vers l'index. On obtient un objet *Series* avec un index hiérarchique à 5 niveaux.

In [ ]:
# transfert des colonnes hiérarchiques vers l'index


Transfert de tous les index en colonnes. On obtient une table avec 6 colonnes distinctes et une ligne par valeur numérique de débit.

In [ ]:
# reset de l'index


In [ ]:
# accès à une ligne particulière


## 7.3 Fusion d'objets de type *DataFrame*

Dans cette section nous allons fusionner le *DataFrame* des communes avec celui des données sur le haut débit. En utilisant l'index "*Code INSEE*" comme pivot.

La méthode **join()** permet de fusionner 2 objets de type *DataFrame* selon l'index (par défaut) ou selon une colonne.

Il existe une option "*how*" pour spécifier comment s'effectue la jointure (terme utilisé dans les base de données) :
- left (par défaut) : utilise l'index du *DataFrame* dont on invoque la méthode
- right : utilise l'index du *DataFrame* passé en argument
- outer : forme l'union des index
- inner : utilise l'intersection des index

Il existe aussi les options *lsuffix* et *rsuffix* pour suffixer les colonnes dupliquées entre *DataFrame*.

In [32]:
alldata = geo.join(thd, rsuffix="_")
alldata.head()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,...,Câble 1 Mbit,Câble 3 Mbit,Câble 8 Mbit,Câble 30 Mbit,Câble 100 Mbit,Fibre 1 Mbit,Fibre 3 Mbit,Fibre 8 Mbit,Fibre 30 Mbit,Fibre 100 Mbit
Code INSEE,,,,,,,,,,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,15.65,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,9.12,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",...,0.0,0.0,0.0,0.0,0.0,0.571,0.571,0.571,0.571,0.571
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,24.48,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,16.05,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",...,0.0,0.0,0.0,0.0,0.0,0.004,0.004,0.004,0.004,0.004
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,6.02,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",...,0.0,0.0,0.0,0.0,0.0,0.934,0.934,0.934,0.934,0.934


In [33]:
alldata.columns

Index(['Code Postal', 'Commune', 'Département', 'Région', 'Statut',
       'Altitude Moyenne', 'Superficie', 'Population', 'geo_point_2d',
       'geo_shape', 'Latitude', 'Longitude', 'Densité', 'Département_',
       'Commune_', '1 Mbit', '3 Mbit', '8 Mbit', '30 Mbit', '100 Mbit',
       'DSL 1 Mbit', 'DSL 3 Mbit', 'DSL 8 Mbit', 'DSL 30 Mbit', 'DSL 100 Mbit',
       'Câble 1 Mbit', 'Câble 3 Mbit', 'Câble 8 Mbit', 'Câble 30 Mbit',
       'Câble 100 Mbit', 'Fibre 1 Mbit', 'Fibre 3 Mbit', 'Fibre 8 Mbit',
       'Fibre 30 Mbit', 'Fibre 100 Mbit'],
      dtype='object')

On constate que *pandas* a suffixé les 2 colonnes "Département" et "Commune" du *DataFrame* de gauche.

Il existe plusieurs autres fonctions de fusion de *DataFrame*.

- fonction **merge()** : jointure sur index ou colonne
- méthode **append()** : ajout d'un objet *Series* ou *DataFrame*
- fonction **concat()** : concaténation de *DataFrames* selon un axe

Voir la documentation : http://pandas.pydata.org/pandas-docs/stable/merging.html

**Exercice**

Construire une table pivot avec les données allant du 1 Mbit au 100 Mbit ventilées par statut des communes.

In [39]:
# on pivote sur les valeurs de la colonne "Statut"
alldata.pivot_table(index='Statut', values=['1 Mbit', '3 Mbit', '8 Mbit', '30 Mbit', '100 Mbit'])

,1 Mbit,100 Mbit,3 Mbit,30 Mbit,8 Mbit
Statut,,,,,
Commune simple,0.951,0.018,0.580,0.112,0.417
Chef-lieu canton,0.993,0.060,0.908,0.411,0.826
Sous-préfecture,0.997,0.110,0.957,0.376,0.883
Préfecture,1.000,0.313,0.957,0.485,0.875
Préfecture de région,1.000,0.474,0.980,0.717,0.924
Capitale d'état,NaN,NaN,NaN,NaN,NaN


## 7.4 Techniques pour compléter les données manquantes

Le *DataFrame* fait apparaître des lignes incomplètes.

In [40]:
alldata[alldata.isnull().any(axis=1)]

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,...,Câble 1 Mbit,Câble 3 Mbit,Câble 8 Mbit,Câble 30 Mbit,Câble 100 Mbit,Fibre 1 Mbit,Fibre 3 Mbit,Fibre 8 Mbit,Fibre 30 Mbit,Fibre 100 Mbit
Code INSEE,,,,,,,,,,,,,,,,,,,,,
09304,09240,SUZAN,ARIEGE,MIDI-PYRENEES,Commune simple,548.0,2.40,0.0,"43.0294117749, 1.43489468125","{""type"": ""Polygon"", ""coordinates"": [[[1.432086...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13201,13001,MARSEILLE--1ER-ARRONDISSEMENT,BOUCHES-DU-RHONE,PROVENCE-ALPES-COTE D'AZUR,Préfecture de région,29.0,1.78,40.9,"43.2999009436, 5.38227869795","{""type"": ""Polygon"", ""coordinates"": [[[5.372144...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13202,13002,MARSEILLE--2E--ARRONDISSEMENT,BOUCHES-DU-RHONE,PROVENCE-ALPES-COTE D'AZUR,Commune simple,6.0,3.64,25.7,"43.3126964178, 5.36364983265","{""type"": ""Polygon"", ""coordinates"": [[[5.373616...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13203,13003,MARSEILLE--3E--ARRONDISSEMENT,BOUCHES-DU-RHONE,PROVENCE-ALPES-COTE D'AZUR,Commune simple,25.0,2.54,45.4,"43.3121200046, 5.38010981423","{""type"": ""Polygon"", ""coordinates"": [[[5.375021...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13204,13004,MARSEILLE--4E--ARRONDISSEMENT,BOUCHES-DU-RHONE,PROVENCE-ALPES-COTE D'AZUR,Commune simple,58.0,2.88,47.1,"43.306733355, 5.40087488752","{""type"": ""Polygon"", ""coordinates"": [[[5.408970...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13205,13005,MARSEILLE--5E--ARRONDISSEMENT,BOUCHES-DU-RHONE,PROVENCE-ALPES-COTE D'AZUR,Commune simple,35.0,2.18,44.5,"43.2928061953, 5.3975770959","{""type"": ""Polygon"", ""coordinates"": [[[5.391944...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13206,13006,MARSEILLE--6E--ARRONDISSEMENT,BOUCHES-DU-RHONE,PROVENCE-ALPES-COTE D'AZUR,Commune simple,41.0,2.06,43.3,"43.2870989546, 5.38104785394","{""type"": ""Polygon"", ""coordinates"": [[[5.373816...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13207,13007,MARSEILLE--7E--ARRONDISSEMENT,BOUCHES-DU-RHONE,PROVENCE-ALPES-COTE D'AZUR,Commune simple,35.0,5.84,35.9,"43.28251778, 5.36323440435","{""type"": ""Polygon"", ""coordinates"": [[[5.367986...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


Nous allons étudier les différentes manières de compléter automatiquement les données manquantes.

In [44]:
# on remet les statuts en type str
alldata2 = alldata.copy().sort_index()
alldata2["Statut"] = alldata2["Statut"].astype(str)
alldata2

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,...,Câble 1 Mbit,Câble 3 Mbit,Câble 8 Mbit,Câble 30 Mbit,Câble 100 Mbit,Fibre 1 Mbit,Fibre 3 Mbit,Fibre 8 Mbit,Fibre 30 Mbit,Fibre 100 Mbit
Code INSEE,,,,,,,,,,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,15.65,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,9.12,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",...,0.0,0.0,0.0,0.0,0.0,0.571,0.571,0.571,0.571,0.571
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,24.48,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,16.05,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",...,0.0,0.0,0.0,0.0,0.0,0.004,0.004,0.004,0.004,0.004
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,6.02,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",...,0.0,0.0,0.0,0.0,0.0,0.934,0.934,0.934,0.934,0.934
01007,01500,AMBRONAY,AIN,RHONE-ALPES,Commune simple,309.0,33.59,2.3,"46.0055913782, 5.35760660735","{""type"": ""Polygon"", ""coordinates"": [[[5.413533...",...,0.0,0.0,0.0,0.0,0.0,0.713,0.713,0.713,0.713,0.713
01008,01500,AMBUTRIX,AIN,RHONE-ALPES,Commune simple,274.0,5.18,0.7,"45.9367134524, 5.3328092349","{""type"": ""Polygon"", ""coordinates"": [[[5.321986...",...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01009,01300,ANDERT-ET-CONDON,AIN,RHONE-ALPES,Commune simple,294.0,6.96,0.3,"45.7873565333, 5.65788307924","{""type"": ""Polygon"", ""coordinates"": [[[5.656393...",...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


Il est possible de remplacer les *NaN* :
- par une valeur donnée,
- par la dernière valeur connue, option *method='ffill'* (forward fill)
- par la prochaine valeur connue, option *method='bfill'* (backward fill)
- par une fonction appliquée à certaines colonnes
- par une fonction d'interpolation sur l'index

In [43]:
# on remplace les NaN par une valeur quelconque
alldata2.fillna(0)

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,...,Câble 1 Mbit,Câble 3 Mbit,Câble 8 Mbit,Câble 30 Mbit,Câble 100 Mbit,Fibre 1 Mbit,Fibre 3 Mbit,Fibre 8 Mbit,Fibre 30 Mbit,Fibre 100 Mbit
Code INSEE,,,,,,,,,,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,15.65,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,9.12,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",...,0.0,0.0,0.0,0.0,0.0,0.571,0.571,0.571,0.571,0.571
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,24.48,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,16.05,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",...,0.0,0.0,0.0,0.0,0.0,0.004,0.004,0.004,0.004,0.004
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,6.02,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",...,0.0,0.0,0.0,0.0,0.0,0.934,0.934,0.934,0.934,0.934
01007,01500,AMBRONAY,AIN,RHONE-ALPES,Commune simple,309.0,33.59,2.3,"46.0055913782, 5.35760660735","{""type"": ""Polygon"", ""coordinates"": [[[5.413533...",...,0.0,0.0,0.0,0.0,0.0,0.713,0.713,0.713,0.713,0.713
01008,01500,AMBUTRIX,AIN,RHONE-ALPES,Commune simple,274.0,5.18,0.7,"45.9367134524, 5.3328092349","{""type"": ""Polygon"", ""coordinates"": [[[5.321986...",...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01009,01300,ANDERT-ET-CONDON,AIN,RHONE-ALPES,Commune simple,294.0,6.96,0.3,"45.7873565333, 5.65788307924","{""type"": ""Polygon"", ""coordinates"": [[[5.656393...",...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [45]:
# on remplace les NaN par la dernière valeur connue
alldata2.fillna(method='ffill')

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,...,Câble 1 Mbit,Câble 3 Mbit,Câble 8 Mbit,Câble 30 Mbit,Câble 100 Mbit,Fibre 1 Mbit,Fibre 3 Mbit,Fibre 8 Mbit,Fibre 30 Mbit,Fibre 100 Mbit
Code INSEE,,,,,,,,,,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,15.65,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,9.12,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",...,0.0,0.0,0.0,0.0,0.0,0.571,0.571,0.571,0.571,0.571
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,24.48,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,16.05,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",...,0.0,0.0,0.0,0.0,0.0,0.004,0.004,0.004,0.004,0.004
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,6.02,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",...,0.0,0.0,0.0,0.0,0.0,0.934,0.934,0.934,0.934,0.934
01007,01500,AMBRONAY,AIN,RHONE-ALPES,Commune simple,309.0,33.59,2.3,"46.0055913782, 5.35760660735","{""type"": ""Polygon"", ""coordinates"": [[[5.413533...",...,0.0,0.0,0.0,0.0,0.0,0.713,0.713,0.713,0.713,0.713
01008,01500,AMBUTRIX,AIN,RHONE-ALPES,Commune simple,274.0,5.18,0.7,"45.9367134524, 5.3328092349","{""type"": ""Polygon"", ""coordinates"": [[[5.321986...",...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01009,01300,ANDERT-ET-CONDON,AIN,RHONE-ALPES,Commune simple,294.0,6.96,0.3,"45.7873565333, 5.65788307924","{""type"": ""Polygon"", ""coordinates"": [[[5.656393...",...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


Il est également possible de supprimer les lignes (*axis=0*) ou les colonnes (*axis=1*) comportant un *NaN*.

In [46]:
# on supprime les lignes avec des valeurs manquantes
alldata2.dropna(axis=0)

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,...,Câble 1 Mbit,Câble 3 Mbit,Câble 8 Mbit,Câble 30 Mbit,Câble 100 Mbit,Fibre 1 Mbit,Fibre 3 Mbit,Fibre 8 Mbit,Fibre 30 Mbit,Fibre 100 Mbit
Code INSEE,,,,,,,,,,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,15.65,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,9.12,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",...,0.0,0.0,0.0,0.0,0.0,0.571,0.571,0.571,0.571,0.571
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,24.48,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,16.05,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",...,0.0,0.0,0.0,0.0,0.0,0.004,0.004,0.004,0.004,0.004
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,6.02,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",...,0.0,0.0,0.0,0.0,0.0,0.934,0.934,0.934,0.934,0.934
01007,01500,AMBRONAY,AIN,RHONE-ALPES,Commune simple,309.0,33.59,2.3,"46.0055913782, 5.35760660735","{""type"": ""Polygon"", ""coordinates"": [[[5.413533...",...,0.0,0.0,0.0,0.0,0.0,0.713,0.713,0.713,0.713,0.713
01008,01500,AMBUTRIX,AIN,RHONE-ALPES,Commune simple,274.0,5.18,0.7,"45.9367134524, 5.3328092349","{""type"": ""Polygon"", ""coordinates"": [[[5.321986...",...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01009,01300,ANDERT-ET-CONDON,AIN,RHONE-ALPES,Commune simple,294.0,6.96,0.3,"45.7873565333, 5.65788307924","{""type"": ""Polygon"", ""coordinates"": [[[5.656393...",...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


Les fonctions de lecture de données comportent également des options permettant de gérer l'absence de valeur.

## 7.5 Techniques pour supprimer les doublons

La méthode **duplicated()** permet de trouver les lignes dupliquées dans un *DataFrame* tandis que la méthode **drop_duplicates()** permet d'en supprimer les lignes dupliquées.

In [47]:
var = thd.reset_index().loc[:, "1 Mbit":"100 Mbit"]
var

,1 Mbit,3 Mbit,8 Mbit,30 Mbit,100 Mbit
0,1.000,0.448,0.052,0.000,0.000
1,0.676,0.594,0.571,0.571,0.571
2,1.000,0.966,0.794,0.234,0.000
3,1.000,0.990,0.942,0.667,0.004
4,1.000,1.000,1.000,0.934,0.934
5,0.973,0.779,0.772,0.713,0.713
6,1.000,0.557,0.013,0.000,0.000
7,1.000,0.616,0.584,0.000,0.000
...,...,...,...,...,...
36685,0.996,0.871,0.624,0.394,0.000


In [48]:
var.duplicated().value_counts()

False    26871
True      9822
dtype: int64

In [49]:
var.drop_duplicates()

,1 Mbit,3 Mbit,8 Mbit,30 Mbit,100 Mbit
0,1.000,0.448,0.052,0.000,0.000
1,0.676,0.594,0.571,0.571,0.571
2,1.000,0.966,0.794,0.234,0.000
3,1.000,0.990,0.942,0.667,0.004
4,1.000,1.000,1.000,0.934,0.934
5,0.973,0.779,0.772,0.713,0.713
6,1.000,0.557,0.013,0.000,0.000
7,1.000,0.616,0.584,0.000,0.000
...,...,...,...,...,...
36685,0.996,0.871,0.624,0.394,0.000
